In [2]:
from pyspark.sql import SparkSession, DataFrame

In [3]:
spark = (
            SparkSession.builder.master("local[*]")
            .appName("SparkByExamples.com")
            .config("spark.jars", "/opt/spark/spark-3.5.0-bin-hadoop3/jars/postgresql-42.7.1.jar")
            .getOrCreate()
)

23/12/20 14:40:22 WARN Utils: Your hostname, trungLap resolves to a loopback address: 127.0.1.1; using 192.168.2.232 instead (on interface wlo1)
23/12/20 14:40:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/20 14:40:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/20 14:40:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
df = (
        spark.read.format("csv")
        .options(header=True, inferSchema=False)
        .load("/home/trungnothot/Downloads/taxi_zones.csv")
    )

In [4]:
POSTGRES_HOST = "localhost"
POSTGRES_PORT = 5432
POSTGRES_DB = "trip_record"
POSTGRES_USER = "admin"
POSTGRES_PASSWORD = "admin123"

jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}") \
    .option("dbtable", "public.warehouse_payment") \
    .option("user", POSTGRES_USER) \
    .option("password", POSTGRES_PASSWORD) \
    .load()

In [5]:
jdbcDF.show()

+------------------+-------+---------------------+------------+----------+-----+------------------+-----------------+------------------+--------------------+-----------+-----------+
|       fare_amount|mta_tax|improvement_surcharge|payment_type|RatecodeID|extra|        tip_amount|     tolls_amount|      total_amount|congestion_surcharge|airport_fee|  PaymentID|
+------------------+-------+---------------------+------------+----------+-----+------------------+-----------------+------------------+--------------------+-----------+-----------+
| 7.900000095367432|    0.5|                  1.0|         1.0|       1.0|  1.0|2.5799999237060547|              0.0|15.479999542236328|                 2.5|       NULL| Y202301290|
|              17.0|    0.5|                  1.0|         2.0|       1.0|  1.0|               0.0|              0.0|              22.0|                 2.5|       NULL| Y202301291|
| 35.20000076293945|    0.5|                  1.0|         2.0|       1.0|  1.0|          

In [8]:
POSTGRES_HOST = "localhost"
POSTGRES_PORT = 5432
POSTGRES_DB = "trip_record"
POSTGRES_USER = "admin"
POSTGRES_PASSWORD = "admin123"
df.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}") \
    .option("dbtable", "public.test") \
    .option("user", POSTGRES_USER) \
    .option("password", POSTGRES_PASSWORD) \
    .save()

In [26]:
df = (
        spark.read.format("parquet")
        .options(header=True, inferSchema=False)
        .load("/home/trung/Downloads/a/green_tripdata_2023-02.parquet")
    )
# count row have tpep_pickup_datetime null
print(df.filter(df.lpep_pickup_datetime.isNull()).count())
# count row have tpep_dropoff_datetime null
print(df.filter(df.lpep_dropoff_datetime.isNull()).count())
# count row have PULocationID null
print(df.filter(df.PULocationID.isNull()).count())
# count row have DOLocationID null
print(df.filter(df.DOLocationID.isNull()).count())

0
0
0
0


In [22]:
df.count()

2913955

In [19]:
# count row have dispatching_base_num null
print(df.filter(df.dispatching_base_num.isNull()).count())
# count row have affiliated_base_number null
print(df.filter(df.Affiliated_base_number.isNull()).count())
# count row have sr_flag null
print(df.filter(df.SR_Flag.isNull()).count())

0
0
1110797


In [29]:
# count row have passenger_count null
print(df.filter(df.passenger_count.isNull()).count())
# count row have trip_distance null
print(df.filter(df.trip_distance.isNull()).count())
# count row have store_and_fwd_flag null
print(df.filter(df.store_and_fwd_flag.isNull()).count())
# count row have trip_type null
print(df.filter(df.trip_type.isNull()).count())
# count row have VendorID null
print(df.filter(df.VendorID.isNull()).count())


4821
0
4821
4826
0


passenger count, store_and_fwd_flag

In [28]:
# count row have fare_amount null
print(df.filter(df.fare_amount.isNull()).count())
# count row have mta_tax null
print(df.filter(df.mta_tax.isNull()).count())
# count row have improvement_surcharge null
print(df.filter(df.improvement_surcharge.isNull()).count())
# count row have payment_type null
print(df.filter(df.payment_type.isNull()).count())
# count row have RatecodeID null
print(df.filter(df.RatecodeID.isNull()).count())
# count row have tip_amount null
print(df.filter(df.tip_amount.isNull()).count())
# count row have tolls_amount null
print(df.filter(df.tolls_amount.isNull()).count())
# count row have total_amount null
print(df.filter(df.total_amount.isNull()).count())
# count row have congestion_surcharge null
print(df.filter(df.congestion_surcharge.isNull()).count())



0
0
0
4821
4821
0
0
0
4821


payment_type, RatecodeID, congestion_surcharge

In [9]:
df = (
        spark.read.format("parquet")
        .options(header=True, inferSchema=False)
        # .schema(trip_schema)
        .load("/home/trung/Downloads/gold_pickup")
    )
#print count null row in PULocationID
print(df.count())
df.filter(df.PULocationID.isNull()).count()

17273410


2914436

In [29]:
gdo = (
    spark.read.format("parquet")
    .options(header=True, inferSchema=False)
    .load("/home/trung/Downloads/gold_dropoff")
)

In [28]:
kdvd =(
    spark.read.format("parquet")
    .options(header=True, inferSchema=False)
    .load("/home/trung/Downloads/taxi_zones.parquet")
)
kdvd.show()

+----------+------------------+------------------+
|LocationID|         longitude|          latitude|
+----------+------------------+------------------+
|         1|-74.17400156582248| 40.69183016020959|
|         2|-73.83129979354713| 40.61674619937379|
|         3| -73.8474217852696| 40.86447372906584|
|         4|-73.97696827424141| 40.72375208451233|
|         5|-74.18848459794721| 40.55265878064343|
|         6|-74.07177024696533|40.600324409468406|
|         7|-73.91969433569462|40.761492617043125|
|         8|-73.92308626324494| 40.77855862576789|
|         9|-73.78794875477833| 40.75103435668664|
|        10|-73.79098676199028| 40.67895308442328|
|        11|-74.00748784386448| 40.60427268170344|
|        12|-74.01556349991374| 40.70294582144012|
|        13|-74.01607927269924|40.712037924667264|
|        14|    -74.0298925099| 40.62483367229869|
|        15|-73.78597285841018|  40.7833330046879|
|        16|-73.77342112932584| 40.76273753216493|
|        17|-73.94990480444068|

In [35]:
gdo.join(kdvd, gdo.DOLocationID == kdvd.LocationID, "left").drop(kdvd.LocationID)
gdo = gdo.withColumn("DOLocationID", gdo["DOLocationID"].cast("int"))
gdo.show()

+-------------------+------------+--------------------+
|   Dropoff_datetime|DOLocationID|           DropOffID|
+-------------------+------------+--------------------+
|2023-01-22 07:49:01|          75|Y2023012225769803776|
|2023-01-22 07:56:41|         236|Y2023012225769803777|
|2023-01-22 07:54:23|         143|Y2023012225769803778|
|2023-01-22 07:56:39|          42|Y2023012225769803779|
|2023-01-22 07:31:54|          79|Y2023012225769803780|
|2023-01-22 07:51:12|         144|Y2023012225769803781|
|2023-01-22 07:06:24|         151|Y2023012225769803782|
|2023-01-22 07:56:49|          48|Y2023012225769803783|
|2023-01-22 07:42:28|         262|Y2023012225769803784|
|2023-01-22 07:52:45|         158|Y2023012225769803785|
|2023-01-22 07:12:28|         246|Y2023012225769803786|
|2023-01-22 07:52:17|         143|Y2023012225769803787|
|2023-01-22 08:04:53|          90|Y2023012225769803788|
|2023-01-22 07:27:46|          50|Y2023012225769803789|
|2023-01-22 07:25:06|         151|Y2023012225769